In [ ]:
# Unfinished Down Centre Handler
%matplotlib inline

import matplotlib.pyplot as plt
from IPython.core.debugger import Pdb; pdb = Pdb()


def plot_trend(y):
    x = list(range(0, len(y)))
    gg = [min(y[1], y[3])] * len(y)
    dd = [max(y[2], y[4])] * len(y)

    plt.figure(figsize=(len(y),4))
    plt.grid()
    plt.plot(x, y)
    plt.plot(x, gg, '--')
    plt.plot(x, dd, '--')
    sx, sy = unfinished_dc_expand_spliter(y)
    plt.plot(sx, sy)
    cs, cgg, cdd = get_last_centre_interval(sx, sy, y)
    plt.plot(cs, cgg, '--')
    plt.plot(cs, cdd, '--')
    plt.show()
    
def get_last_centre_interval(sx, sy, ts):
    if len(sx) == 3 or len(sx) == 5 or len(sx) == 6:
        return [], [], []
    elif len(sx) == 4:
        s0 = sx[2]
        s1 = sy[2]
        e0 = sx[3]
        e1 = sy[3]
        if e0 - s0 == 3:
            return [s0 + 1, e0 - 1], [ts[s0+1]] * 2, [ts[e0-1]] * 2
        elif e0 - s0 >= 5:
            gg = min(ts[s0+1], ts[s0+3])
            dd = max(ts[s0+2], ts[s0+4])
            num = e0 - s0 - 1
            return list(range(s0 + 1, e0)), [gg] * num, [dd] * num
        else:
            raise RuntimeError('Not Handled')
    else:
        raise RuntimeError('Not Handled')

def get_dc_high_order(ts):
    ths_start_offset = 3
    ths_end_offset = ts[-1][0] - 6
    
    ths = ts[ths_start_offset : ths_end_offset + 1][::2]
    
    ths_ordered = sorted(ths, key=lambda x: x[1], reverse=True)
    return ths_ordered

def get_dc_low_order(ts, th):
    tl_start_offset = th[0] + 3
    tl_end_offset = ts[-1][0] - 3
    
    tls = ts[tl_start_offset : tl_end_offset + 1][::2]
    
    tls_ordered = sorted(tls, key=lambda x: x[1])
    return tls_ordered

def finished_dc_expand_spliter(t_list):
    ts = list(enumerate(t_list))
    th_ordered = get_dc_high_order(ts)
    tl_ordered = get_dc_low_order(ts, th_ordered[0])
    
    th = th_ordered[0]
    tl = tl_ordered[0]
    
    found = False
    for tl in tl_ordered:
        if ts[-1][0] - tl[0] <= 5:
            sx = [ts[0][0], th[0], tl[0], ts[-1][0]]
            sy = [ts[0][1], th[1], tl[1], ts[-1][1]]
            found = True
            break
        else:
            i = tl[0]
            dd = max(ts[i+2][1], ts[i+4][1])
            found_centre = True
            for zn in ts[(i+5):-2][::2]:
                if zn[1] < dd:
                    found_centre = False
                    break
            if found_centre:
                sx = [ts[0][0], th[0], tl[0], ts[-1][0]]
                sy = [ts[0][1], th[1], tl[1], ts[-1][1]]
                found = True
                break
    
    if found:
        if sx[1] - sx[0] >= 11:
            t_slice = t_list[sx[0] : sx[1] + 1]
            sxx, syy = finished_dc_expand_spliter(t_slice)
            sxx = [x + sx[0] for x in sxx]
            sx = sxx + sx[2:]
            sy = [t_list[x] for x in sx]
        elif sx[3] - sx[2] >= 11:
            t_slice = t_list[sx[2] : sx[3] + 1]
            sxx, syy = finished_dc_expand_spliter(t_slice)
            sxx = [x + sx[2] for x in sxx]
            sx = sx[:2] + sxx
            sy = [t_list[x] for x in sx]
        return sx, sy
    else:   
        raise RuntimeError('Not Handled')
    
def unfinished_dc_expand_spliter(t_list):
    ts = list(enumerate(t_list))
    
    found_down = False
    for i in range(ts[-2][0], ts[4][0] - 1, -2):
        if ts[i][1] < ts[i-2][1] and ts[i-1][1] < ts[i-3][1]:
            found_down = True
            continue
        else:
            break

    if found_down:
        th = ts[i-1]
        if th[0] >= 11:
            t_slice = t_list[:th[0]+1]
            sx, sy = finished_dc_expand_spliter(t_slice)
            sx.append(ts[-2][0])
            sy.append(ts[-2][1])
            return sx, sy
        else:
            return [ts[0][0], th[0], ts[-2][0]], [ts[0][1], th[1], ts[-2][1]]
    else:
        sx, sy = finished_dc_expand_spliter(t_list)
        if sy[-1] > sy[-2]:
            return sx, sy
        else:
            th_start_offset = sx[-4] + 3
            th_end_offset = sx[-1] - 4

            ths = ts[th_start_offset : th_end_offset + 1][::2]

            ths_max = max(ths, key=lambda x: x[1])
            sx = sx[0: -3] + [ths_max[0], ts[-2][0]]
            sy = [t_list[x] for x in sx]
            return sx, sy
        

ys = []

ys.append(([0, 100, 60, 120, 70, 110, 25, 155, 35, 145, 45, 150], []))
ys.append(([0, 100, 60, 120, 70, 110, 25, 155, 35, 145, 45, 135], []))
ys.append(([0, 100, 60, 120, 70, 110, 40, 130, 55, 140, 65, 75], []))
ys.append(([0, 100, 60, 120, 70, 110, 40, 130, 80, 140, 65, 75], []))
ys.append(([0, 100, 60, 120, 70, 110, 25, 155, 35, 145, 45, 115, 55, 105], []))

ys.append(([0, 100, 60, 120, 70, 110, 35, 75, 25, 145, 65, 85], []))
ys.append(([0, 100, 50, 120, 60, 90, 30, 110, 80, 130, 45, 70, 35, 105], []))
ys.append(([0, 100, 50, 130, 60, 90, 30, 110, 80, 120, 45, 70, 35, 105], []))
ys.append(([0, 100, 50, 120, 60, 90, 30, 70, 40, 110, 80, 130, 45, 70, 35, 105], []))
ys.append(([0, 100, 50, 130, 60, 90, 30, 70, 40, 110, 80, 120, 45, 70, 35, 105], []))

ys.append(([0, 100, 60, 120, 70, 110, 25, 155, 55, 145, 45, 150], []))
ys.append(([0, 100, 60, 120, 70, 110, 40, 130, 80, 140, 65, 75], []))
ys.append(([0, 100, 60, 120, 70, 110, 40, 140, 80, 130, 65, 75], []))

ys.append(([0, 110, 70, 100, 60, 80, 61, 94, 77, 98, 70, 136, 68, 90, 66, 82, 73, 110], []))
ys.append(([0, 110, 70, 100, 60, 100, 78, 90, 53, 109, 56, 141, 99, 106, 89, 99, 93, 141], []))
ys.append(([0, 100, 70, 110, 60, 100, 53, 90, 78, 109, 56, 109, 99, 106, 89, 99, 93, 141], []))

ys.append(([0, 100, 60, 110, 70, 120, 80, 130, 90, 140, 50, 75], []))

for y in ys:
    plot_trend(y[0])


In [ ]:
# Random Centre Generator
%matplotlib inline

import random
import matplotlib.pyplot as plt

y_max = 150
y_min = 50
num_max = 18

def generate_next(y_list, direction):
    if direction == 1:
        y_list.append(random.randint(max(y_list[2], y_list[4], y_list[-1]) + 1, y_max))
    elif direction == -1:
        y_list.append(random.randint(y_min, min(y_list[1], y_list[3], y_list[-1]) - 1))

# y_base = [0, 100, 60, 110, 70]
y_base = [0, 110, 70, 100, 60]
# y_base = [0, 100, 60, 90, 70]
# y_base = [0, 90, 70, 100, 60]

direction = 1
for i in range(5, num_max):
    generate_next(y_base, direction)
    direction = 0 - direction

y_base[-1] = 110
print(y_base)
for i in range(11, len(y_base), 2):
    y = y_base[:(i + 1)]
    plot_trend(y)
    

In [19]:
%matplotlib inline

import matplotlib.pyplot as plt


ys = []

# Group 1
# ys.append([0, 100, 60, 110, 70, 99, 66, 121, 91, 141, 57, 111, 69, 111])
# ys.append([0, 100, 60, 110, 70, 105, 58, 102, 74, 137, 87, 142, 55, 128])
# ys.append([0, 100, 60, 110, 70, 115, 75, 120, 80, 125, 85, 130, 90, 135])
# ys.append([0, 100, 60, 110, 70, 120, 80, 130, 90, 140, 50, 75])
# ys.append([0, 100, 60, 110, 70, 114, 52, 75, 54, 77, 65, 100, 66, 87, 70, 116])
# ys.append([0, 100, 60, 110, 70, 72, 61, 143, 77, 91, 82, 100, 83, 124, 89, 99, 89, 105])

# Group 2
# ys.append([0, 110, 70, 100, 60, 142, 51, 93, 78, 109, 60, 116, 50, 106])
# ys.append([0, 110, 70, 100, 60, 88, 70, 128, 82, 125, 72, 80, 63, 119])
# ys.append([0, 110, 70, 100, 60, 74, 66, 86, 57, 143, 50, 95, 70, 91])
# ys.append([0, 110, 70, 100, 60, 77, 73, 122, 96, 116, 82, 124, 69, 129])
# ys.append([0, 110, 70, 100, 60, 147, 53, 120, 77, 103, 56, 76, 74, 92])
# ys.append([0, 110, 70, 100, 60, 95, 55, 90, 50, 85, 45, 80, 40, 75])
# ys.append([0, 110, 70, 100, 60, 100, 78, 90, 53, 109, 56, 141, 99, 106, 89, 99, 93, 141])

# Group 3
# ys.append([0, 100, 60, 90, 70, 107, 55, 123, 79, 112, 64, 85, 74, 110])
# ys.append([0, 100, 60, 90, 70, 77, 55, 107, 76, 141, 87, 91, 60, 83])
# ys.append([0, 100, 60, 90, 70, 114, 67, 93, 58, 134, 53, 138, 64, 107])
# ys.append([0, 100, 60, 90, 70, 77, 66, 84, 79, 108, 87, 107, 72, 89])
# ys.append([0, 100, 60, 90, 70, 88, 72, 86, 74, 84, 76, 82, 74, 80])

# Group 4
# ys.append([0, 90, 70, 100, 60, 131, 57, 144, 85, 109, 82, 124, 87, 101])
# ys.append([0, 90, 70, 100, 60, 150, 56, 112, 63, 95, 84, 118, 58, 110])
# ys.append([0, 90, 70, 100, 60, 145, 64, 112, 69, 86, 71, 119, 54, 95])
# ys.append([0, 90, 70, 100, 60, 105, 55, 110, 50, 115, 45, 120, 40, 125])

for y_base in ys:
    for i in range(11, len(y_base), 2):
        y = y_base[:(i + 1)]
        plot_trend(y)
    